# 6章
- 以下で実行するコードには確率的な処理が含まれていることがあり、コードの出力結果と本書に記載されている出力例が異なることがあります。

In [1]:
# chap6ディレクトリを作成して、移動する。
!mkdir chap6
%cd ./chap6

/Users/harukikondo/git/bert-book/chap6


In [2]:
# 必要なライブラリをインポートする。
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0 pytorch-lightning==1.2.7

     |████████████████████████████████| 830 kB 8.3 MB/s 
     |████████████████████████████████| 234 kB 85.9 MB/s 
     |████████████████████████████████| 6.0 MB 55.0 MB/s 
     |████████████████████████████████| 648 kB 92.3 MB/s 
     |████████████████████████████████| 781 kB 68.0 MB/s 
     |████████████████████████████████| 132 kB 54.7 MB/s 
     |████████████████████████████████| 97 kB 26.6 MB/s 
     |████████████████████████████████| 3.5 MB 60.0 MB/s 
     |████████████████████████████████| 146 kB 49.6 MB/s 
     |████████████████████████████████| 49 kB 11.4 MB/s 
     |████████████████████████████████| 124 kB 71.4 MB/s 
You should consider upgrading via the '/Users/harukikondo/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [3]:
# 必要なライブラリをインポーナイ座ー
import random
import glob
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForSequenceClassification
import pytorch_lightning as pl

# 日本語の事前学習モデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

In [ ]:
# トークナイザーを用意する。
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
# 文章分類用のモデルをロードする。
bert_sc = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
# GPUに配置する。
bert_sc = bert_sc.cuda()

In [ ]:
# 文章のデータセット
text_list = [
    "この映画は面白かった。",
    "この映画の最後にはがっかりさせられた。",
    "この映画を見て幸せな気持ちになった。"
]
# 判定のデータセット
label_list = [1,0,1]

# データの符号化
encoding = tokenizer(text_list, padding = 'longest', return_tensors='pt')
encoding = { k: v.cuda() for k, v in encoding.items() }
# テンソル化する。
labels = torch.tensor(label_list).cuda()

# 推論
with torch.no_grad():
    # BERTモデルに入力する。
    output = bert_sc.forward(**encoding)
# 分類スコアを取得する。
scores = output.logits 
# スコアが最も高いラベル
labels_predicted = scores.argmax(-1) 
# 正解数
num_correct = (labels_predicted==labels).sum().item() 
# 精度
accuracy = num_correct/labels.size(0) 
# 結果を出力する。
print("# scores:")
print(scores.size())
print("# predicted labels:")
print(labels_predicted)
print("# accuracy:")
print(accuracy)

In [ ]:
# 符号化
encoding = tokenizer(text_list, padding='longest', return_tensors='pt') 
# 入力にラベルを加える。
encoding['labels'] = torch.tensor(label_list) 
encoding = { k: v.cuda() for k, v in encoding.items() }

# ロスの計算
output = bert_sc(**encoding)
# 損失の取得
loss = output.loss 
print(loss)

In [5]:
# 6-7
#データのダウンロード
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz 
#ファイルの解凍
!tar -zxf ldcc-20140209.tar.gz 

zsh:1: command not found: wget
tar: Error opening archive: Failed to open 'ldcc-20140209.tar.gz'


In [6]:
# 6-8
!cat ./text/it-life-hack/it-life-hack-6342280.txt # ファイルを表示

http://news.livedoor.com/article/detail/6342280/
2012-03-06T13:00:00+0900
USB3.0対応で爆速データ転送！　9倍速のリーダー／ライター登場
USB3.0が登場してから今年で4年目となるがパソコン側でのUSB3.0ポート搭載が進んで来ても対応機器がなかなか充実していない現状がある。そんな中で新しく高速な読み取りが可能なメモリーカードリーダー／ライターが登場した。

バッファローコクヨサプライがUSB3.0対応のカードリーダー／ライターを発表した。SDHC対応のSD系メディアやコンパクトフラッシュ、メモリースティック系メディア、xDピクチャーカードといったデジカメやスマホ、携帯ゲームといった機器で使われている各種メディアを従来よりも短時間でPCに取り込むことが可能になる。

転送速度が5Gbps（理論値）とUSB2.0の480Mbpsと比べて爆速になったUSB3.0はPC側の対応が進んで来ていたが高速転送が生かせる周辺機器としては、外付けHDDや一部のUSBメモリーくらいしかなかった。これに多くのメディアが扱えるリーダー／ライターが加わることで手軽にUSB3.0の恩恵を受けることができるようになる。

今回発表されたのは、USB3.0ケーブルとカードリーダー本体が分かれるタイプの「BSCR09U3」シリーズ（3,240円）、USB3.0コネクタをカードリーダー本体に内蔵している「BSCRD04U3」シリーズ（2,690円）だ。共にホワイトとブラックのカラーバリエーションが用意される（発売は3月下旬以降）。

■リリースページ
■バッファローコクヨサプライ




■バッファローの記事をもっと見る
・約283gでカバンに入る！小型キーボードの驚くべき機能
・3種類のホットキーで使いやすい！AndroidとPCで使えるキーボードの魅力
・ドラえもんもビックリの新アイテム！マウスとキーボードが合体"OPAir"
・ありそうでなかった便利機能！ファイル仕分けする画期的なHDD


サンディスク SanDisk microSDHC 32GB（microSD 32GB） 超高速クラス4  変換アダプター付 世界国内シェアNo.1 バルク品
クチコミを見る


In [ ]:
# データローダーの作成
dataset_for_loader = [
    {'data':torch.tensor([0,1]), 'labels':torch.tensor(0)},
    {'data':torch.tensor([2,3]), 'labels':torch.tensor(1)},
    {'data':torch.tensor([4,5]), 'labels':torch.tensor(2)},
    {'data':torch.tensor([6,7]), 'labels':torch.tensor(3)},
]
loader = DataLoader(dataset_for_loader, batch_size=2)

# データセットからミニバッチを取り出す
for idx, batch in enumerate(loader):
    print(f'# batch {idx}')
    print(batch)
    ## ファインチューニングではここでミニバッチ毎の処理を行う

In [ ]:
# データセットからランダムにデータを取り出す。
loader = DataLoader(dataset_for_loader, batch_size=2, shuffle=True)
# データセットからミニバッチを取り出す
for idx, batch in enumerate(loader):
    print(f'# batch {idx}')
    print(batch)

In [ ]:
# カテゴリーのリスト
category_list = [
    'dokujo-tsushin',
    'it-life-hack',
    'kaden-channel',
    'livedoor-homme',
    'movie-enter',
    'peachy',
    'smax',
    'sports-watch',
    'topic-news'
]
# トークナイザのロード
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
# 各データの形式を整える
max_length = 128
# データローダーを格納するための配列
dataset_for_loader = []
for label, category in enumerate(tqdm(category_list)):
    for file in glob.glob(f'./text/{category}/{category}*'):
        lines = open(file).read().splitlines()
        # ファイルの4行目からを抜き出す。
        text = '\n'.join(lines[3:]) 
        # トークンを符号化する。
        encoding = tokenizer(
            text,
            max_length=max_length, 
            padding='max_length',
            truncation=True
        )
        # ラベルを追加する。
        encoding['labels'] = label 
        encoding = { k: torch.tensor(v) for k, v in encoding.items() }
        # 整形済みのデータを追加する。
        dataset_for_loader.append(encoding)

In [ ]:
# データセットを出力する。
print(dataset_for_loader[0])

In [ ]:
# データセットの分割
# ランダムにシャッフル
random.shuffle(dataset_for_loader) 
n = len(dataset_for_loader)
n_train = int(0.6*n)
n_val = int(0.2*n)
# 学習データ
dataset_train = dataset_for_loader[:n_train] 
# 検証データ
dataset_val = dataset_for_loader[n_train:n_train+n_val] 
# テストデータ
dataset_test = dataset_for_loader[n_train+n_val:] 

# データセットからデータローダを作成
# 学習データはshuffle=Trueにする。
dataloader_train = DataLoader(dataset_train, batch_size=32, shuffle=True) 
dataloader_val = DataLoader(dataset_val, batch_size=256)
dataloader_test = DataLoader(dataset_test, batch_size=256)

In [ ]:
# モデルの振る舞いを定義したクラス
class BertForSequenceClassification_pl(pl.LightningModule):
        
    # model_name: Transformersのモデルの名前
    # num_labels: ラベルの数
    # lr: 学習率
    def __init__(self, model_name, num_labels, lr):

        super().__init__()
        
        # 引数のnum_labelsとlrを保存。
        # 例えば、self.hparams.lrでlrにアクセスできる。
        # チェックポイント作成時にも自動で保存される。
        self.save_hyperparameters() 
        # BERTのロード
        self.bert_sc = BertForSequenceClassification.from_pretrained(model_name,num_labels=num_labels)
        
    # 学習データのミニバッチ(`batch`)が与えられた時に損失を出力する関数を書く。
    # batch_idxはミニバッチの番号であるが今回は使わない。
    def training_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        loss = output.loss
        # 損失を'train_loss'の名前でログをとる。
        self.log('train_loss', loss) 
        return loss
        
    # 検証データのミニバッチが与えられた時に、
    # 検証データを評価する指標を計算する関数を書く。
    def validation_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        val_loss = output.loss
        # 損失を'val_loss'の名前でログをとる。
        self.log('val_loss', val_loss) 

    # テストデータのミニバッチが与えられた時に、
    # テストデータを評価する指標を計算する関数を書く。
    def test_step(self, batch, batch_idx):
        # バッチからラベルを取得
        labels = batch.pop('labels') 
        # BERTに入力する。
        output = self.bert_sc(**batch)
        # もっとも高いスコアのものを取り出す。
        labels_predicted = output.logits.argmax(-1)
        num_correct = ( labels_predicted == labels ).sum().item()
        #精度
        accuracy = num_correct/labels.size(0) 
        # 精度を'accuracy'の名前でログをとる。
        self.log('accuracy', accuracy) 

    # 学習に用いるオプティマイザを返す関数を書く。
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [ ]:
# ファインチューニング用の設定を行う。
# 学習時にモデルの重みを保存する条件を指定
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='model/',
)

# 学習の方法を指定
# gpuを1つ資料する。
# エポック数 10
# モデルの重みを保存するタイミング
trainer = pl.Trainer(
    gpus=1, 
    max_epochs=10,
    callbacks = [checkpoint]
)

In [ ]:
# PyTorch Lightningモデルのロード
model = BertForSequenceClassification_pl(MODEL_NAME, num_labels=9, lr=1e-5)
# ファインチューニングを行う。
trainer.fit(model, dataloader_train, dataloader_val) 

In [ ]:
# ベストモデルのファイル
best_model_path = checkpoint.best_model_path 
print('ベストモデルのファイル: ', checkpoint.best_model_path)
print('ベストモデルの検証データに対する損失: ', checkpoint.best_model_score)

In [ ]:
# 学習結果を出力する。
%load_ext tensorboard
%tensorboard --logdir ./

In [ ]:
# テストデータを渡して検証する。
test = trainer.test(test_dataloaders=dataloader_test)
print(f'Accuracy: {test[0]["accuracy"]:.2f}')

In [ ]:
# ファインチューニングしたモデルを使用する。
# PyTorch Lightningモデルのロード
model = BertForSequenceClassification_pl.load_from_checkpoint(best_model_path) 

# Transformers対応のモデルを./model_transformesに保存
model.bert_sc.save_pretrained('./model_transformers') 

In [ ]:
# ファインチューニングしたモデルのデータを読み込んでモデルをロードする。
bert_sc = BertForSequenceClassification.from_pretrained('./model_transformers')